In [1]:
import os, sys

fileDir = os.getcwd()
path = os.path.join(fileDir, "..")
sys.path.append(path)

from utilities import *

# Sample Data

In [2]:
data = [(1, "Alice"),
        (2, "Bob"),
        (3, "Charlie")]

relationship = [(1, 2), 
                (2, 3)]

columns = ["id", "name"]
relationship_columns = ["from", "to"]

# Neo4j connector test

In [3]:
session = create_spark_session("Neo4j Test", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


## Write

In [9]:
df = session.createDataFrame(data, columns)

options = get_default_options(SparkConnector.NEO4J)
options["node.keys"] = "id"
options["labels"] = ":Person"

spark_write(SparkConnector.NEO4J, df, "Overwrite", options=options)


Dataframe saved to NEO4J


In [10]:
df = session.createDataFrame(relationship, relationship_columns)
options = get_default_options(SparkConnector.NEO4J)
options["relationship"] = "FRIEND"
options["relationship.source.labels"] = "Person"
options["relationship.save.strategy"] = "keys"
options["relationship.source.node.keys"] = "from:id"
options["relationship.target.labels"] = "Person"
options["relationship.target.node.keys"] = "to:id"

spark_write(SparkConnector.NEO4J, df, "Overwrite", options=options)

Dataframe saved to NEO4J


In [5]:
df = session.createDataFrame(relationship, relationship_columns)
options = get_default_options(SparkConnector.NEO4J)
options["query"] = "MATCH (n :Person), (m :Person) \
                    WHERE (n.id = event.from) AND (m.id = event.to) \
                    CREATE (m)-[:FRIEND]->(n)"
spark_write(SparkConnector.NEO4J, df, "Overwrite", options=options)


Dataframe saved to NEO4J


## Read

In [11]:
options = get_default_options(SparkConnector.NEO4J)
options["labels"] = ":Person"

df = spark_read(SparkConnector.NEO4J, session, options=options)
df.show()

Dataframe loaded from neo4j
+----+--------+-------+---+
|<id>|<labels>|   name| id|
+----+--------+-------+---+
|   0|[Person]|  Alice|  1|
|   1|[Person]|Charlie|  3|
|   2|[Person]|    Bob|  2|
+----+--------+-------+---+



In [12]:
options = get_default_options(SparkConnector.NEO4J)
options["query"] = "MATCH (n:Person)-[r:FRIEND]->(m:Person) \
                    RETURN n.name as name,  m.name as friend"

df = spark_read(SparkConnector.NEO4J, session, options=options)
df.show()

Dataframe loaded from neo4j
+-----+-------+
| name| friend|
+-----+-------+
|Alice|    Bob|
|  Bob|Charlie|
+-----+-------+



## Stop spark context and spark session

In [13]:

session.sparkContext.stop()
session.stop()